### Extract Skills from JT Offers descriptions
- In this notebook we will try to extract skills from offers that were matched with ESCO job titles.
- We will have at first a list of skills that exists in the offer
- Another list that ESCO propose for each job
- And a third list which represents the intersection between these two lists

In [67]:
import skillsdb_functions as sf ## This file contain all function used in this notebook

### Get our Data

In [2]:
Skills = sf.retrieve_final_skills()
JtOffers = sf.retrieve_jt_offers()

### We clean our data
- We delete skills with less than two words

In [21]:
flat_list_app = [item for sublist in Skills.French_only_alt_lower_1 for item in sublist]
print("number of appelation before cleaning :",len(list(set(flat_list_app))))

Skills['French_only_alt_lower_1_clean'] = Skills.French_only_alt_lower_1.apply(sf.nettoyage_skills)
flat_list_app = [item for sublist in Skills.French_only_alt_lower_1_clean for item in sublist]
print("number of appelation after cleaning :",len(list(set(flat_list_app))))

Skills = Skills[Skills['preferredLabel_y'] != "Windows Phone"]
Skills.at[int(Skills.index[Skills['preferredLabel_y'] == "ML (programmation informatique)"].values)\
,'French_only_alt_lower_1_clean'] = ['machine learning']
Skills.at[int(Skills.index[Skills['preferredLabel_y'] == "traitement automatique du langage naturel"].values)\
,'French_only_alt_lower_1_clean'] = ['nlp']
Skills = Skills[Skills['preferredLabel_y'] != "ski"]
Skills = Skills[Skills['preferredLabel_y'] != "Sass"]
Skills = Skills[Skills['preferredLabel_y'] != 'id Tech']
### Take only Jt offers that have ESCO jobs
JtOffers_matched = JtOffers[JtOffers['ESCO'].apply(lambda x:len(x)>0)]


number of appelation before cleaning : 68507
number of appelation after cleaning : 56800


#### Add spaces to skills to match them with entities in text that do not start in the middle of a word

In [72]:
def add_space_list(list_alt):
    return [sf.add_space(m) for m in list_alt]

Skills.French_only_alt_lower_1_clean.head().apply(add_space_list)

0    [ coordon le tach personnel musical ,  ger per...
1    [ ger le procedur penitentiair ,  supervis le ...
2    [ appliqu pratiqu non oppress ,  agir manier n...
3    [ control respect reglement materiel roul ,  c...
4    [ determin le servic disponibl ,  etabl servic...
Name: French_only_alt_lower_1_clean, dtype: object

In [73]:
Skills['French_only_alt_lower_1_clean'] = Skills.French_only_alt_lower_1_clean.apply(add_space_list)

# We start our matching between ESCO skills and JT offers

## Web Dev
- We get all offers matched from JT offers with the ESCO job **Web Dev** (get_desc_esco())
- We join all descriptions from these offers into one big string
- We Match by key words all the skills from ESCO with the string of description
- The result will be stocked in a dictionary with keys representing **Matched skills** and values representing **frequency of appearance**

In [74]:
### Mathcing 1
### load esco for web dev
job = "développeur web/développeuse web"
WEB_Job = JtOffers_matched[JtOffers_matched['ESCO'].apply(lambda x:sf.get_desc_esco(x,job))]
print("Number of Web developer offers :",len(WEB_Job))

### Matching ESCO JT
desc = " ".join(WEB_Job.description_proc)
Dict_Matched = dict()
_ = Skills.apply(lambda row:sf.matching_with_apply(row['preferredLabel_y'],row['French_only_alt_lower_1_clean'],desc,Dict_Matched),axis=1)

Number of Web developer offers : 5957


## 1-List of skills found in Web dev Jt offers

In [30]:
sorted_web = sorted(Dict_Matched.items(), key=sf.operator.itemgetter(1),reverse=True)
sorted_web

[('Java (programmation informatique)', 3920),
 ('Python (programmation informatique)', 814),
 ('Drupal', 479),
 ('Oracle Data Integrator', 476),
 ('AJAX', 368),
 ('SAS language', 331),
 ('PHP', 292),
 ('PostgreSQL', 281),
 ('TypeScript', 246),
 ('NoSQL', 243),
 ('LESS', 222),
 ('CSS', 186),
 ('modélisation orientée objet', 116),
 ('JavaScript Framework', 111),
 ('Swift (programmation informatique)', 103),
 ('Scala', 85),
 ('gérer une base de données', 74),
 ('déploiement de solutions', 68),
 ('définir des exigences techniques', 67),
 ('Apache Maven', 62),
 ('SPARK', 60),
 ('modèles de données', 51),
 ('rédiger un cahier des charges', 45),
 ('programmation informatique', 42),
 ('collecter les données', 41),
 ('effectuer une analyse de besoins de clients', 39),
 ('Hadoop', 39),
 ('outils de gestion de configuration logicielle', 38),
 ('utiliser des bases de données', 37),
 ('Perl', 36),
 ('travailler de manière autonome', 35),
 ('répondre aux demandes des clients', 33),
 ('APL', 32),
 ('

## 2-List of skills proposed by ESCO for Web dev job

In [33]:
### ESCO
WEB_ESCO = sf.get_skills_job_esco("développeur web/développeuse web")
listOfESCOSkills = WEB_ESCO.keys()
listOfDescSkills = [m[0] for m in sorted_web]
print("Skills from ESCO for Web Dev :",len(listOfESCOSkills))
print("Skills from Desc for Web Dev :",len(listOfDescSkills))
print("Skills from the two :",len(set(listOfDescSkills).intersection(set(listOfESCOSkills))))
print(sorted(listOfESCOSkills))

Skills from ESCO for Web Dev : 86
Skills from Desc for Web Dev : 213
Skills from the two : 31
['ABAP', 'AJAX', 'APL', 'ASP.NET', 'Ansible', 'Apache Maven', 'Assembly (programmation informatique)', 'C#', 'C++', 'COBOL', 'CSS', 'CoffeeScript', 'Common Lisp', 'Eclipse (environnement de développement intégré)', 'Erlang', 'Groovy', 'Haskell', 'JSSS', 'Java (programmation informatique)', 'JavaScript', 'Jenkins (outils de gestion de configuration logicielle)', 'KDevelop', 'LESS', 'Lisp', 'MATLAB', 'ML (programmation informatique)', 'Microsoft Visual C++', 'Objective-C', 'OpenEdge Advanced Business Language', 'PHP', 'Pascal (programmation informatique)', 'Perl', 'Prolog (programmation informatique)', 'Puppet (outils de gestion de configuration logicielle)', 'Python (programmation informatique)', 'R', 'Ruby (programmation informatique)', 'SAP R3', 'SAS language', 'STAF', 'Salt (outils de gestion de configuration logicielle)', 'Sass', 'Scala', 'Scratch (programmation informatique)', 'Smalltalk (

## 3- Intersection Between ESCO and JT offers skills for Web Developer

In [34]:
set(listOfDescSkills).intersection(set(listOfESCOSkills))

{'ABAP',
 'AJAX',
 'APL',
 'Ansible',
 'Apache Maven',
 'COBOL',
 'CSS',
 'CoffeeScript',
 'Erlang',
 'Java (programmation informatique)',
 'JavaScript',
 'KDevelop',
 'LESS',
 'PHP',
 'Perl',
 'Python (programmation informatique)',
 'SAS language',
 'STAF',
 'Scala',
 'Swift (programmation informatique)',
 'TypeScript',
 'VBScript',
 'Visual Basic',
 'concevoir une interface utilisateur',
 'développer un prototype de logiciel',
 'environnement de développement intégré',
 'fournir une documentation technique',
 'modélisation orientée objet',
 'outils de gestion de configuration logicielle',
 'programmation informatique',
 'programmation web'}

### To Save to csv results obtained

In [42]:
import pandas as pd
## to Csv
def check_in_esco(x,lista):
    if x in lista:
        return "OUI"
    return "NON"
Web_df = pd.DataFrame.from_dict(Dict_Matched,orient="index",columns=['Freq']).sort_values(by='Freq',ascending = False)
Web_df['ESCO_Match'] = Web_df.index.map(lambda x:check_in_esco(x,listOfESCOSkills))
print(Web_df.head())
#Web_df.to_csv("Web_df4.csv")
listt = list(set(listOfDescSkills).intersection(set(listOfESCOSkills)))
listo = list(set(WEB_ESCO.keys()) - set(listt))
Web_esco = pd.DataFrame({'ESCO': pd.Series(list(WEB_ESCO.keys()))})

def check_skill_type(x):
    return WEB_ESCO[x]
Web_esco['ESCO/JT'] = Web_esco['ESCO'].apply(lambda x:check_in_esco(x,listt))
Web_esco['ESCO_type'] = Web_esco['ESCO'].apply(check_skill_type)
Web_esco = Web_esco.sort_values(by="ESCO_type")
#Web_esco.to_csv("Web_esco4.csv")

                                     Freq ESCO_Match
Java (programmation informatique)    3920        OUI
Python (programmation informatique)   814        OUI
Drupal                                479        NON
Oracle Data Integrator                476        NON
AJAX                                  368        OUI


## graphiste

In [49]:
### Mathcing 1
### load esco for web dev
job = "graphiste"
Graph_Job = JtOffers_matched[JtOffers_matched['ESCO'].apply(lambda x:sf.get_desc_esco(x,job))]
print("Number of graphiste offers :",len(Graph_Job))

### Matching ESCO JT
desc = " ".join(Graph_Job.description_proc)
Dict_Matched = dict()
_ = Skills.apply(lambda row:sf.matching_with_apply(row['preferredLabel_y'],row['French_only_alt_lower_1_clean'],desc,Dict_Matched),axis=1)
sorted_graph = sorted(Dict_Matched.items(), key=sf.operator.itemgetter(1),reverse=True)
### ESCO
Graph_ESCO = sf.get_skills_job_esco("graphiste")
listOfESCOSkills = Graph_ESCO.keys()
listOfDescSkills = [m[0] for m in sorted_graph]
print("Skills from ESCO for graphiste :",len(listOfESCOSkills))
print("Skills from Desc for graphiste :",len(listOfDescSkills))
print("Skills from the two :",len(set(listOfDescSkills).intersection(set(listOfESCOSkills))))

Number of graphiste offers : 351
Skills from ESCO for graphiste : 50
Skills from Desc for graphiste : 39
Skills from the two : 2


In [50]:
sorted_graph

[('Adobe Illustrator', 69),
 ('Oracle Data Integrator', 30),
 ('Java (programmation informatique)', 12),
 ('STAF', 9),
 ('collecter les données', 7),
 ("respecter les spécifications d'un contrat", 6),
 ('SAS language', 6),
 ('Perl', 5),
 ('LESS', 5),
 ('CSS', 4),
 ('transporter des matériaux de construction', 3),
 ('trouver des solutions aux problèmes', 2),
 ('analyser des demandes d’indemnisation', 2),
 ('travailler de manière autonome', 2),
 ('souci du détail', 2),
 ('avoir un esprit créatif', 2),
 ('optimisation des moteurs de recherche', 2),
 ('logiciel de création', 2),
 ('mobile marketing', 2),
 ('effectuer une analyse de besoins de clients', 1),
 ('utiliser différents moyens de communication', 1),
 ('études de communication', 1),
 ('rechercher de nouvelles idées', 1),
 ('techniques d’impression textile', 1),
 ('pratiquer des sports', 1),
 ('impression offset', 1),
 ('élaborer de nouveaux produits', 1),
 ('examiner la maquette d’une publicité', 1),
 ('développer des idées créativ

## DRH

In [76]:
job = 'directeur des ressources humaines/directrice des ressources humaines'
RH_Job = JtOffers_matched[JtOffers_matched['ESCO'].apply(lambda x:sf.get_desc_esco(x,job))]
print("Number of directeur RH offers :",len(RH_Job))


### Matching ESCO JT
desc = " ".join(RH_Job.description_proc)
Dict_Matched = dict()
_ = Skills.apply(lambda row:sf.matching_with_apply(row['preferredLabel_y'],row['French_only_alt_lower_1_clean'],desc,Dict_Matched),axis=1)
sorted_drh = sorted(Dict_Matched.items(), key=sf.operator.itemgetter(1),reverse=True)
### ESCO
DRH_ESCO = sf.get_skills_job_esco("directeur des ressources humaines/directrice des ressources humaines")
listOfESCOSkills = DRH_ESCO.keys()
listOfDescSkills = [m[0] for m in sorted_drh]

print("Skills from ESCO for DRH :",len(listOfESCOSkills))
print("Skills from Desc for DRH :",len(listOfDescSkills))
print("Skills from the two :",len(set(listOfDescSkills).intersection(set(listOfESCOSkills))))

Number of directeur RH offers : 633
Skills from ESCO for DRH : 137
Skills from Desc for DRH : 35
Skills from the two : 7


/Users/alabayoudh/skillsdb_functions.py:393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  Skills_for_job = df_relation[df_relation['occupationUri'] == url][['skillUri','relationType']]


In [77]:
sorted_drh

[('gestion des ressources humaines', 142),
 ('respecter des obligations légales', 10),
 ('hygiène et sécurité sur le lieu de travail', 9),
 ('gérer des budgets', 6),
 ("appliquer les politiques d'une entreprise", 5),
 ('négocier des contrats de travail', 5),
 ('interagir verbalement en anglais', 4),
 ('gestion du personnel', 3),
 ('philosophies d’amélioration continue', 2),
 ("élaborer des politiques d'emploi", 2),
 ('équipements de protection individuelle', 2),
 ('porter un équipement de protection contre les bruits industriels', 2),
 ('assurer le respect de politiques', 2),
 ('assurer le respect des exigences légales', 2),
 ('gérer une base de données', 2),
 ('technologie informatique', 2),
 ('acheter des stocks de matières premières', 1),
 ('participer à des réunions de conception', 1),
 ('répondre aux exigences des pouvoirs publics', 1),
 ('gérer des situations de soins d’urgence', 1),
 ('faire preuve de diplomatie', 1),
 ('gérer la communication entre des équipes', 1),
 ('conseill

## We see that there's some technical skills that appears in jobs and do not count as skills
- This is probably due to fact that technical skills generally are composed of less characters and sometimes expressed in abbreviations, that's why they match with jobs such as HR
- An idea that came to me is to add a function that matches skills without including the tech skills
- The problem was that ESCO do not differentiate between tech and non tech skills, so I needed to make an hypothesis after consulting the data
- Generally tech skills don't have alternative appellations, so a way to do is to match skills that have at least two appellations other than the preferred label.

In [59]:
job = 'directeur des ressources humaines/directrice des ressources humaines'
RH_Job = JtOffers_matched[JtOffers_matched['ESCO'].apply(lambda x:sf.get_desc_esco(x,job))]
print("Number of directeur RH offers :",len(RH_Job))


### Matching ESCO JT
desc = " ".join(RH_Job.description_proc)
Dict_Matched = dict()
_ = Skills.apply(lambda row:sf.matching_with_apply_tech(row['preferredLabel_y'],row['French_only_alt_lower_1_clean'],desc,Dict_Matched,False),axis=1)
sorted_drh = sorted(Dict_Matched.items(), key=sf.operator.itemgetter(1),reverse=True)
### ESCO
DRH_ESCO = sf.get_skills_job_esco("directeur des ressources humaines/directrice des ressources humaines")
listOfESCOSkills = DRH_ESCO.keys()
listOfDescSkills = [m[0] for m in sorted_drh]

print("Skills from ESCO for DRH :",len(listOfESCOSkills))
print("Skills from Desc for DRH :",len(listOfDescSkills))
print("Skills from the two :",len(set(listOfDescSkills).intersection(set(listOfESCOSkills))))

Number of directeur RH offers : 633
Skills from ESCO for DRH : 137
Skills from Desc for DRH : 35
Skills from the two : 5


/Users/alabayoudh/skillsdb_functions.py:393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  Skills_fil['relationType'] = Skills_fil.loc[:,'conceptUri'].apply(lambda x:Skills_for_job[x])


In [60]:
sorted_drh

[('gestion des ressources humaines', 150),
 ('respecter des obligations légales', 10),
 ('hygiène et sécurité sur le lieu de travail', 9),
 ("appliquer les politiques d'une entreprise", 5),
 ('négocier des contrats de travail', 5),
 ('interagir verbalement en anglais', 4),
 ('gestion du personnel', 3),
 ('philosophies d’amélioration continue', 2),
 ("élaborer des politiques d'emploi", 2),
 ('réaliser des services de recrutement', 2),
 ('équipements de protection individuelle', 2),
 ('porter un équipement de protection contre les bruits industriels', 2),
 ('assurer le respect de politiques', 2),
 ('assurer le respect des exigences légales', 2),
 ("faire preuve d'enthousiasme", 2),
 ('gérer une base de données', 2),
 ('concevoir un système d’information', 2),
 ('technologie informatique', 2),
 ('travailler dans un environnement international', 1),
 ('acheter des stocks de matières premières', 1),
 ('participer à des réunions de conception', 1),
 ('répondre aux exigences des pouvoirs publ